In [ ]:
# default_exp models._vae

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#export

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import re

In [ ]:
#export

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class UnFlatten(nn.Module):
    def forward(self, input):
        return input.view(-1, 64, 7, 7)

In [ ]:
#export

class VAE(nn.Module):
    def __init__(self, latent_size):
        super(VAE, self).__init__()
        
        self.latent_size = latent_size
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 4, 1, 2),
            nn.ReLU(True),
            nn.Conv2d(32, 32, 4, 2, 1),
            nn.ReLU(True),
            nn.Conv2d(32, 64, 4, 2, 1),
            nn.ReLU(),
            Flatten()
        )
        
        self.mu = nn.Linear(64*7*7, latent_size)
        self.logvar = nn.Linear(64*7*7, latent_size)
        
        self.upsample = nn.Linear(latent_size, 64*7*7)
        self.decoder = nn.Sequential(
            UnFlatten(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 32, 4, 2, 1, 1),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 1, 4, 1 , 2),
            nn.Sigmoid()
        )
    
    def bottleneck(self, h):
        mu, logvar = self.mu(h), self.logvar(h)
        z = self.reparametrize(mu, logvar)
        return z, mu, logvar
    
    def encode(self, x):
        h = self.encoder(x)
        z, mu, logvar = self.bottleneck(h)
        return z, mu, logvar
    
    def decode(self, z):
        z = self.upsample(z)
        z = self.decoder(z)
        return z
        
    def reparametrize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar) + 1e-8
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu
    
    def forward(self, x):
        z, mu, logvar = self.encode(x)
        z = self.decode(z)
        return z, mu, logvar

In [ ]:
#export

def vae_loss(x_pred, x_orig):
    
    x, z_mu, z_var = x_pred

    recon_loss = F.binary_cross_entropy(x, x_orig, reduction='sum')
    
    # KL divergence loss
    KLD = -0.5 * torch.sum(1 + z_var - z_mu.pow(2) - z_var.exp())
    
    # total_loss
    loss = recon_loss + KLD
    
    return loss

In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_CIFAR.ipynb.
Converted 01_simpleCNN.ipynb.
Converted 02_ResNet.ipynb.
Converted 03_transformations.ipynb.
Converted 04_vae.ipynb.
Converted index.ipynb.
Converted NLP - Introduction.ipynb.
Converted NLP part 2.ipynb.
Converted VAE-Introduction.ipynb.
Converted Word embeddings - 2.ipynb.
This cell doesn't have an export destination and was ignored:
e
This cell doesn't have an export destination and was ignored:
e
Converted Word embeddings.ipynb.
